# ANLI with LLM

You have to implement in this notebook a better ANLI classifier using an LLM.
This classifier must be implemented using DSPy.


In [53]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
from dotenv import load_dotenv
import os
import dspy
load_dotenv("grok_key.ini") 
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])

# for ollama 
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

Step 1: deine the reward function and metrics

In [54]:
from typing import Literal

## Implement the DSPy classifier program.

from sentence_transformers import SentenceTransformer, util
import dspy

# Load the Sentence-Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2') # the most cost effecitve model according to sbert documentation.

# Calculate the semantic similarity between two strings
def semantic_similarity(premise_hypothesis: str, reason: str) -> float:
    embeddings1 = model.encode(premise_hypothesis, convert_to_tensor=True)
    embeddings2 = model.encode(reason, convert_to_tensor=True)
    # The item() method returns a single value from a tensor
    return util.pytorch_cos_sim(embeddings1, embeddings2).item()

# Define a reward function that checks for semantic similarity
def reward_semantic_similarity(args: dict, pred: dspy.Prediction) -> float:
    # Access the 'premise' and 'hypothesis' from the input arguments dictionary
    premise = args['premise']
    hypothesis = args['hypothesis']

    # Access the 'reason' from the prediction object
    reason = pred.reason

    # Call the semantic_similarity helper function with the correct arguments
    return semantic_similarity(f"{premise}, {hypothesis}", reason)

Step 2: define the modules and signatures using Refine and the reward function.

IMPORTANT NOTE: we picked a heuristical threshold which is popular to use for similarity check.
This threshold acts as a soft filter - as the module will select the prediction with the highest reward anyway (and specificaly when the reward score is smaller than the threshold). See https://dspy.ai/api/modules/Refine/ for reference.

In [55]:

# -------------------------
# Signatures
# -------------------------
from tqdm import tqdm


threshold = 0.6 # popular heurisitic threshold for semantic similarity, according to ChatGPT, Gemini, Perplexity, Claude.

class NLISignatureJoint(dspy.Signature):
    """
    Classify the relationship between the premise and hypothesis 
    into: entailment, neutral, or contradiction.
    Provide a justification for your answer.
    """
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    reason: str = dspy.OutputField()
    label: Literal['entailment', 'neutral', 'contradiction'] = dspy.OutputField()


class NLISignatureReasonFirst(dspy.Signature):
    """
    First, reason about the relationship between the premise and hypothesis.
    """
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    reason: str = dspy.OutputField()


class NLISignatureLabelSecond(dspy.Signature):
    """
    Given the premise, hypothesis, and a reasoning explanation,
    classify the relationship into: entailment, neutral, or contradiction.
    """
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    reason: str = dspy.InputField()
    label: Literal['entailment', 'neutral', 'contradiction'] = dspy.OutputField()


# -------------------------
# Helper for batching
# -------------------------

def batch_predict(predictor, examples, batch_size=20, num_threads=8, desc="Processing"):
    results = []
    for i in tqdm(range(0, len(examples), batch_size), desc=desc):
        sub_batch = examples[i:i + batch_size]
        results.extend(
            predictor.batch(sub_batch, num_threads=num_threads)
        )
    return results


# -------------------------
# Joint Prompt Classifier
# -------------------------

class NLIJointClassifier(dspy.Module):
    """
    Runs the joint-prompt model: predicts both reason and label in one step.
    """
    def __init__(self, joint_predictor, batch_size=20, num_threads=8):
        super().__init__()
        self.joint_predictor = dspy.Refine(
            module=joint_predictor,
            N=3,  # Try up to 2 times
            reward_fn=reward_semantic_similarity,
            threshold=threshold  # default
        )
        self.batch_size = batch_size
        self.num_threads = num_threads

    def forward(self, examples):
        return batch_predict(
            self.joint_predictor, examples,
            batch_size=self.batch_size, num_threads=self.num_threads,
            desc="Joint Prompt Processing"
        )


# -------------------------
# Pipeline Classifier
# -------------------------

class NLIPipelineClassifier(dspy.Module):
    """
    Runs the reasoning-first → label-second pipeline.
    """
    def __init__(self, reason_predictor, label_predictor, batch_size=20, num_threads=8):
        super().__init__()
        self.reason_predictor = dspy.Refine(
            module=reason_predictor,
            N=3,  # Try up to 2 times
            reward_fn=reward_semantic_similarity,
            threshold=threshold # default
        )
        self.label_predictor = label_predictor
        self.batch_size = batch_size
        self.num_threads = num_threads

    def forward(self, examples):
        # Stage 1: Reasoning
        reasoning_results = batch_predict(
            self.reason_predictor, examples,
            batch_size=self.batch_size, num_threads=self.num_threads,
            desc="Stage 1: Reasoning"
        )

        # Build label stage input
        label_inputs = [
            dspy.Example(
                premise=ex.premise,
                hypothesis=ex.hypothesis,
                reason=reasoning_results[i].reason
            ).with_inputs("premise", "hypothesis", "reason")
            for i, ex in enumerate(examples)
        ]

        # Stage 2: Label prediction
        label_results = batch_predict(
            self.label_predictor, label_inputs,
            batch_size=self.batch_size, num_threads=self.num_threads,
            desc="Stage 2: Labeling"
        )

        # Attach reasoning to final predictions
        for i, pred in enumerate(label_results):
            pred.reason = reasoning_results[i].reason

        return label_results


## Load ANLI dataset

In [56]:
from datasets import load_dataset

dataset = load_dataset("facebook/anli")
dataset = dataset.filter(lambda x: x['reason'] != None and x['reason'] != "")

## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [57]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [58]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

Step 3: prepare the data and the classifying programs, and define a metric computation function

In [59]:
# prepare the training set
preprocessed_examples_for_joint = [
    dspy.Example(
        premise=row["premise"],
        hypothesis=row["hypothesis"],
        reason=row["reason"],
        label=row["label"]
    ).with_inputs("premise", "hypothesis")
    for row in dataset['dev_r3']
]

preprocessed_joint_no_reason = [
    dspy.Example(
        premise=row["premise"],
        hypothesis=row["hypothesis"],
    ).with_inputs("premise", "hypothesis")
    for row in dataset['dev_r3']
]


preprocessed_examples_for_pipeline = [
    dspy.Example(
        premise=row["premise"],
        hypothesis=row["hypothesis"],
        reason=row["reason"],
        label=row["label"]
    ).with_inputs("premise", "hypothesis")
    for row in dataset['dev_r3'] 
]

preprocessed_examples_for_pipeline_no_reason = [
    dspy.Example(
        premise=row["premise"],
        hypothesis=row["hypothesis"],
    ).with_inputs("premise", "hypothesis")
    for row in dataset['dev_r3']
]


# JOINT PROMPT
joint_predictor = dspy.ChainOfThought(NLISignatureJoint)
joint_program = NLIJointClassifier(joint_predictor)


# PIPELINE
reason_predictor = dspy.ChainOfThought(NLISignatureReasonFirst) # CoT needed only for reasoning
label_predictor = dspy.Predict(NLISignatureLabelSecond)
pipeline_program = NLIPipelineClassifier(
    reason_predictor=reason_predictor,
    label_predictor=label_predictor
)


def compute_metrics(preds, golds):
    return {
        "accuracy": accuracy.compute(predictions=preds, references=golds)["accuracy"],
        "precision": precision.compute(predictions=preds, references=golds, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=golds, average="macro")["recall"],
        "f1": f1.compute(predictions=preds, references=golds, average="macro")["f1"],
    }

Step 4: calculate the results

In [60]:
NEW_TEST_SIZE = 1200 # THE NUMBER OF ITEMS IN DEV_R3

print("\n\nRunning joint-prompt model on the test set...")
results_joint = joint_program(preprocessed_joint_no_reason[:NEW_TEST_SIZE])

print("\n\nRunning pipeline model on the test set...")
results_pipeline = pipeline_program(preprocessed_examples_for_pipeline[:NEW_TEST_SIZE])



Running joint-prompt model on the test set...


Joint Prompt Processing:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\wexle\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Processed 20 / 20 examples: 100%|██████████| 20/20 [00:01<00:00, 13.14it/s]

...

Stage 2: Labeling:  97%|█████████▋| 58/60 [16:06<00:37, 18.93s/it]


Processed 20 / 20 examples: 100%|██████████| 20/20 [00:17<00:00,  1.12it/s]

Stage 2: Labeling:  98%|█████████▊| 59/60 [16:24<00:18, 18.61s/it]


Processed 20 / 20 examples: 100%|██████████| 20/20 [00:14<00:00,  1.38it/s]

Stage 2: Labeling: 100%|██████████| 60/60 [16:38<00:00, 16.65s/it]

Step 5: display the Metrics results

In [61]:
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}
pred_labels_joint = [label2id[pred.label] for pred in results_joint]
gold_labels_joint = [ex.label for ex in preprocessed_examples_for_joint[:NEW_TEST_SIZE]]
metrics_joint = compute_metrics(pred_labels_joint, gold_labels_joint)

# Print the metrics
print("\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print("JOINT Model scores:")
print(f"F1 score: {metrics_joint['f1']:.4f}")
print(f"Accuracy: {metrics_joint['accuracy']:.4f}")
print(f"Precision: {metrics_joint['precision']:.4f}")
print(f"Recall: {metrics_joint['recall']:.4f}")
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n")

#######################################################################

pred_labels_pipeline = [label2id[pred.label] for pred in results_pipeline]
gold_labels_pipeline = [ex.label for ex in preprocessed_examples_for_pipeline[:NEW_TEST_SIZE]]
metrics_pipeline = compute_metrics(pred_labels_pipeline, gold_labels_pipeline)

# Print the metrics
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print("PIPELINE Model scores:")
print(f"F1 score: {metrics_pipeline['f1']:.4f}")
print(f"Accuracy: {metrics_pipeline['accuracy']:.4f}")
print(f"Precision: {metrics_pipeline['precision']:.4f}")
print(f"Recall: {metrics_pipeline['recall']:.4f}")
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")



@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
JOINT Model scores:
F1 score: 0.6967
Accuracy: 0.6917
Precision: 0.7223
Recall: 0.6916
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
PIPELINE Model scores:
F1 score: 0.7004
Accuracy: 0.7017
Precision: 0.6996
Recall: 0.7017
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


Step 6: define a function to calculate and present the Semantic similarity scores across the 3 passages and two models

In [62]:
def present_semantic_similarity(pairs: list[dspy.Example], predictions: list[dspy.Example]) -> None:
    """
    Computes and displays semantic similarity statistics between (premise, hypothesis) pairs
    and the generated reasons from a model.
    
    Args:
        pairs: List of input dspy.Example objects containing 'premise' and 'hypothesis'.
        predictions: List of output dspy.Example objects containing 'reason'.
    """
    assert len(pairs) == len(predictions), "Input and output lists must have the same length."

    if len(pairs[0]) == 1:
        scores = [
            semantic_similarity(f"{pair.reason}", pred.reason)
            for pair, pred in zip(pairs, predictions)
        ]
    else:
        scores = [
            semantic_similarity(f"{pair.premise}, {pair.hypothesis}", pred.reason)
            for pair, pred in zip(pairs, predictions)
        ]

    above_threshold_count = sum(score >= threshold for score in scores)
    print(f"the threshold is: {threshold}")
    print(f"Number of pairs with score above threshold ({threshold}): {above_threshold_count} out of {len(pairs)}")
    print(f"Percentage of pairs with score above threshold:{above_threshold_count / len(pairs) * 100:.2f}%")

    average_score = sum(scores) / len(scores)
    print(f"Average semantic similarity score for model: {average_score:.4f}")
    above_average_count = sum(score >= average_score for score in scores)
    print(f"Number of pairs with score above average ({average_score:.4f}): {above_average_count} out of {len(pairs)}")
    print(f"Percentage of pairs with score above average:{above_average_count / len(pairs) * 100:.2f}%")


Step 7: Perform the semantic similarity calculation on the three passages

In [63]:
######## PASSAGE 1 ##########
# HUMAN reason vs. LLM reason
human_examples = [dspy.Example(reason=row["reason"]) for row in dataset['dev_r3']][:NEW_TEST_SIZE] 
print("\n\n@@@ Calculating semantic similarity of HUMAN reason vs JOINT LLM reason...@@@")
present_semantic_similarity(human_examples, results_joint)
print("\n\n@@@ Calculating semantic similarity for the HUMAN reason vs PIPELINE LLM reason...@@@")
present_semantic_similarity(human_examples, results_pipeline)

######## PASSAGE 2 ##########
# (premise, hypothesis) pairs and the generated reasons from the model
print("\n\n@@@ Calculating semantic similarity for the (premise, hypothesis) pairs vs JOINT results...@@@")
present_semantic_similarity(preprocessed_joint_no_reason[:NEW_TEST_SIZE], results_joint)
print("\n\n@@@ Calculating semantic similarity for the (premise, hypothesis) pairs vs PIPELINE results...@@@")
present_semantic_similarity(preprocessed_examples_for_pipeline_no_reason[:NEW_TEST_SIZE], results_pipeline)


######## PASSAGE 3 ##########
# (premise, hypothesis) pairs and the reasons provided from the model
human_predictions = [dspy.Prediction(reason=row["reason"]) for row in dataset['dev_r3']][:NEW_TEST_SIZE]  # Use the first 40 examples for human predictions
print("\n\n@@@ Calculating semantic similarity for the (premise, hypothesis) pairs vs HUMAN REASON results... @@@")
present_semantic_similarity(preprocessed_examples_for_joint[:NEW_TEST_SIZE], human_predictions) # no importance for joint\pipeline here, both have the same dataset in this case





@@@ Calculating semantic similarity of HUMAN reason vs JOINT LLM reason...@@@


c:\Users\wexle\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


the threshold is: 0.6
Number of pairs with score above threshold (0.6): 559 out of 1200
Percentage of pairs with score above threshold:46.58%
Average semantic similarity score for model: 0.5569
Number of pairs with score above average (0.5569): 649 out of 1200
Percentage of pairs with score above average:54.08%


@@@ Calculating semantic similarity for the HUMAN reason vs PIPELINE LLM reason...@@@
the threshold is: 0.6
Number of pairs with score above threshold (0.6): 505 out of 1200
Percentage of pairs with score above threshold:42.08%
Average semantic similarity score for model: 0.5368
Number of pairs with score above average (0.5368): 649 out of 1200
Percentage of pairs with score above average:54.08%


@@@ Calculating semantic similarity for the (premise, hypothesis) pairs vs JOINT results...@@@
the threshold is: 0.6
Number of pairs with score above threshold (0.6): 720 out of 1200
Percentage of pairs with score above threshold:60.00%
Average semantic similarity score for model: 0.

Step 8: Compute the agreement between the models
NOTE: MODEL 1 = JOINT , MODEL 2 = PIPELINE

In [64]:
# MODEL 1 == JOINT
# MODEL 2 == PIPELINE
gold_labels = [row.label for row in preprocessed_examples_for_joint] # no importance, both preprocessed contain the same data
TEST_SIZE = len(results_joint)

# on how many samples both models are correct
correct_predictions = 0
for i in range(TEST_SIZE):
    if (label2id[results_joint[i].label] == gold_labels[i]) and (label2id[results_pipeline[i].label] == gold_labels[i]):
        correct_predictions += 1
print(f"Both models are correct on {correct_predictions} out of {TEST_SIZE} samples.")
print(f"Both models are correct on {correct_predictions / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples llm is correct and DeBERTa_v3_ is incorrect
only_joint_correct = 0
for i in range(TEST_SIZE):
    if (label2id[results_joint[i].label] == gold_labels[i]) and (label2id[results_pipeline[i].label] != gold_labels[i]):
        only_joint_correct += 1
print(f"only JOINT correct on {only_joint_correct} out of {TEST_SIZE} samples.")
print(f"only JOINT correct on {only_joint_correct / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


# On how many samples DeBERTa_v3 is correct and llm is incorrect
only_pipeline_correct = 0
for i in range(TEST_SIZE):
    if (label2id[results_joint[i].label] != gold_labels[i]) and (label2id[results_pipeline[i].label] == gold_labels[i]):
        only_pipeline_correct += 1
print(f"only PIPELINE correct on {only_pipeline_correct} out of {TEST_SIZE} samples.")
print(f"only PIPELINE correct on {only_pipeline_correct / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


# on how many samples both models are incorrect
both_wrong = 0
for i in range(TEST_SIZE):
    if (label2id[results_joint[i].label] != gold_labels[i]) and (label2id[results_pipeline[i].label] != gold_labels[i]):
        both_wrong += 1
print(f"both WRONG on {both_wrong} out of {TEST_SIZE} samples.")
print(f"both WRONG on {both_wrong / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

Both models are correct on 724 out of 1200 samples.
Both models are correct on 60.33% of the samples.


only JOINT correct on 106 out of 1200 samples.
only JOINT correct on 8.83% of the samples.


only PIPELINE correct on 118 out of 1200 samples.
only PIPELINE correct on 9.83% of the samples.


both WRONG on 252 out of 1200 samples.
both WRONG on 21.00% of the samples.




STEP 9: Compute the agreement between DeBERTa_v3 and the JOINT, PROMPT models

In [ ]:
# DeBERTa_v3 vs JOINT
%store -r pred_test_r3
optimized_llm_predictions = results_joint
non_llm_predictions = pred_test_r3 # DeBERTa_v3_predictions
TEST_SIZE = len(optimized_llm_predictions)
# gold_labels = [label2id[row['gold_label']] for row in pred_test_r3]


# on how many samples both models are correct
correct_predictions = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == row['gold_label']) and (row['gold_label'] == row['pred_label'])
)

print(f"Both models are correct on {correct_predictions} out of {TEST_SIZE} samples.")
print(f"Both models are correct on {correct_predictions / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples llm is correct and DeBERTa_v3_ is incorrect
llm_correct_deberta_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == row['gold_label']) and (row['gold_label'] != row['pred_label'])
)
print(f"LLM JOINT is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect} out of {TEST_SIZE} samples.")
print(f"LLM JOINT is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples DeBERTa_v3 is correct and llm is incorrect
deberta_correct_llm_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (row['pred_label'] == row['gold_label']) and (llm_pred.label != row['gold_label'])
)
print(f"DeBERTa_v3 is correct and LLM JOINT is incorrect on {deberta_correct_llm_incorrect} out of {TEST_SIZE} samples.")
print(f"DeBERTa_v3 is correct and LLM JOINT is incorrect on {deberta_correct_llm_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


# on how many samples both models are incorrect
both_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label != row['gold_label']) and (row['pred_label'] != row['gold_label'])
)
print(f"Both models are incorrect on {both_incorrect} out of {TEST_SIZE} samples.")
print(f"Both models are incorrect on {both_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

Both models are correct on 282 out of 1200 samples.
Both models are correct on 23.50% of the samples.


LLM JOINT is correct and DeBERTa_v3 is incorrect on 357 out of 1200 samples.
LLM JOINT is correct and DeBERTa_v3 is incorrect on 29.75% of the samples.


DeBERTa_v3 is correct and LLM JOINT is incorrect on 295 out of 1200 samples.
DeBERTa_v3 is correct and LLM JOINT is incorrect on 24.58% of the samples.


Both models are incorrect on 266 out of 1200 samples.
Both models are incorrect on 22.17% of the samples.




In [66]:
# DeBERTa_v3 vs PIPELINE
%store -r pred_test_r3
optimized_llm_predictions = results_pipeline
non_llm_predictions = pred_test_r3 # DeBERTa_v3_predictions
TEST_SIZE = len(optimized_llm_predictions)
# gold_labels = [label2id[row['gold_label']] for row in pred_test_r3]


# on how many samples both models are correct
correct_predictions = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == row['gold_label']) and (row['gold_label'] == row['pred_label'])
)

print(f"Both models are correct on {correct_predictions} out of {TEST_SIZE} samples.")
print(f"Both models are correct on {correct_predictions / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples llm is correct and DeBERTa_v3_ is incorrect
llm_correct_deberta_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == row['gold_label']) and (row['gold_label'] != row['pred_label'])
)
print(f"LLM PIPELINE is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect} out of {TEST_SIZE} samples.")
print(f"LLM PIPELINE is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples DeBERTa_v3 is correct and llm is incorrect
deberta_correct_llm_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (row['pred_label'] == row['gold_label']) and (llm_pred.label != row['gold_label'])
)
print(f"DeBERTa_v3 is correct and LLM PIPELINE is incorrect on {deberta_correct_llm_incorrect} out of {TEST_SIZE} samples.")
print(f"DeBERTa_v3 is correct and LLM PIPELINE is incorrect on {deberta_correct_llm_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


# on how many samples both models are incorrect
both_incorrect = sum(
    1 for row, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label != row['gold_label']) and (row['pred_label'] != row['gold_label'])
)
print(f"Both models are incorrect on {both_incorrect} out of {TEST_SIZE} samples.")
print(f"Both models are incorrect on {both_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

Both models are correct on 321 out of 1200 samples.
Both models are correct on 26.75% of the samples.


LLM PIPELINE is correct and DeBERTa_v3 is incorrect on 334 out of 1200 samples.
LLM PIPELINE is correct and DeBERTa_v3 is incorrect on 27.83% of the samples.


DeBERTa_v3 is correct and LLM PIPELINE is incorrect on 256 out of 1200 samples.
DeBERTa_v3 is correct and LLM PIPELINE is incorrect on 21.33% of the samples.


Both models are incorrect on 289 out of 1200 samples.
Both models are incorrect on 24.08% of the samples.


